In [ ]:
# imports
import os 
import requests
from dotenv import load_dotenv
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openAI = OpenAI()


In [ ]:

system_prompt = "You are an agent that specializes in answering questions related to software development, code, LLMs, AI and programming. Give precise answers to user's questions. Respond in clearly formatted markdown"

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def get_user_prompt(question):
    user_prompt = f"""
    Answer the following question
    {question}
    """
    return user_prompt
    

In [ ]:
# Get answer using gpt-4o-mini, with streaming

def answer_gpt_with_streaming(question):
    stream = openAI.chat.completions.create(model=MODEL_GPT, messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content" : get_user_prompt(question)},
    ],
    stream=True)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


In [ ]:
answer_gpt_with_streaming(question)

In [ ]:
# Get answer using Ollama running llama3.2

def answer_ollama(question):
    LLAMA_ENDPOINT_URL = "http://localhost:11434/api/chat"
    HEADERS = {"Content-Type": "application/json"}
    payload = {
        "model": MODEL_LLAMA,
        "messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": get_user_prompt(question)} ],
        "stream": False
    }
    
    response = requests.post(LLAMA_ENDPOINT_URL, json=payload, headers=HEADERS)
    display(Markdown(response.json()['message']['content'])) 




In [ ]:
answer_ollama(question)